## Develop this to have entrylist with highest entry lowest and average

## maybe entry from the average trade script

In [58]:
import numpy as np 
import pandas as pd 
from datetime import date
#Data Source (Pandas Data Reader)
from pandas_datareader import data
import requests 
import json
from dateutil.relativedelta import relativedelta


#Data Source(Pandas Data Reader)


In [59]:
#entrylist =[['ANKR/USDT',0.07684],['AVAX/USDT',57],['WAXP/USDT',0.3818],['REEF/USDT',0.01346],['LINK/USDT',14.15],['AR/USDT',30],['ATOM/USDT',23],['MIR/USDT',2.343],['ALGO/USDT',0.909],['ZRX/USDT',0.90],['XTZ/USDT',2.856],['KSM/USDT',163],]
#entrylist =[['ANKRUSDT',0.07684],['AVAXUSDT',57],['WAXPUSDT',0.3818],['REEFUSDT',0.01131],['LINKUSDT',14.15],['ARUSDT',30],['ATOMUSDT',23],['MIRUSDT',2.343],['ALGOUSDT',0.909],['ZRXUSDT',0.60],['XTZUSDT',2.856],['KSMUSDT',163],['NEARUSDT',14],['WINUSDT',0.0003138],['CELRUSDT',0.05063],['DENTUSDT',0.002459],['LRCUSDT',0.7047],['CRVUSDT',3.529],['IMXUSDT',2.709],['SCRTUSDT',5],['RENUSDT',0.2547],['SUSHIUSDT',4.522],['CAKEUSDT',11.24],['XLMUSDT',0.2109],['HNTUSDT',30],['OMGUSDT',4.5],['LINAUSDT',0.024],['MIRUSDT',2.343],['FLUXUSDT',1.523]]
entrylist =[['ANKRUSDT',0.05415],['AVAXUSDT',57],['WAXPUSDT',0.3174],['REEFUSDT',0.00930],['LINKUSDT',14.15],['ARUSDT',30],['ATOMUSDT',23],['MIRUSDT',2.343],['ALGOUSDT',0.909],['ZRXUSDT',0.50],['XTZUSDT',2.856],['KSMUSDT',150],['NEARUSDT',10.320],['WINUSDT',0.0003138],['DENTUSDT',0.002459],['LRCUSDT',0.677],['CRVUSDT',2.12],['IMXUSDT',1.30],['SCRTUSDT',5],['RENUSDT',0.3047],['SUSHIUSDT',3.5],['CAKEUSDT',11.24],['XLMUSDT',0.2109],['HNTUSDT',30],['OMGUSDT',4.5],['LINAUSDT',0.01759],['FLUXUSDT',1.523],['SRMUSDT',1.8],['VTHOUSDT',0.0031],['ANTUSDT',5.616],['RAYUSDT',2.5],['ROSEUSDT',0.24055],['MOVRUSDT',51.6],['TFUELUSDT',0.22],['GLMRUSDT',4.5],['COTIUSDT',0.2277],['ONEUSDT',0.13171],['VETUSDT',0.05138],['RUNEUSDT',3.709],['CELRUSDT',0.034063],['ONTUSDT',0.4652],['GALAUSDT',0.24794],['GRTUSDT',0.3164],['DOTUSDT',16.31],['ZILUSDT',0.04410],['FILUSDT',20],['SCUSDT',0.015],['MATICUSDT',1.4],['FTMUSDT',1.0],['BANDUSDT',3.6],['1INCHUSDT',1.6],['AUDIOUSDT',1.0]]
exposurelist =[['ANKRUSDT',311],['AVAXUSDT',2850],['WAXPUSDT',527],['REEFUSDT',603],['LINKUSDT',6000],['ARUSDT',2450],['ATOMUSDT',1863],['MIRUSDT',85],['ALGOUSDT',2735],['ZRXUSDT',245],['XTZUSDT',1200],['KSMUSDT',7800],['NEARUSDT',1360],['WINUSDT',285],['DENTUSDT',385],['LRCUSDT',660],['CRVUSDT',323],['IMXUSDT',269],['SCRTUSDT',601],['RENUSDT',684],['SUSHIUSDT',2730],['CAKEUSDT',4000],['XLMUSDT',368],['HNTUSDT',999],['OMGUSDT',765],['LINAUSDT',205],['FLUXUSDT',224],['SRMUSDT',587],['VTHOUSDT',413],['ANTUSDT',200],['RAYUSDT',100],['ROSEUSDT',634],['MOVRUSDT',500],['TFUELUSDT',244],['GLMRUSDT',400],['COTIUSDT',500],['ONEUSDT',1000],['VETUSDT',500],['RUNEUSDT',750],['CELRUSDT',500],['ONTUSDT',400],['GALAUSDT',650],['GRTUSDT',1700],['DOTUSDT',10000],['ZILUSDT',1000],['FILUSDT',2000],['SCUSDT',200],['MATICUSDT',8000],['FTMUSDT',2000],['BANDUSDT',750],['1INCHUSDT',1000],['AUDIOUSDT',700]]
startdate = '2021-01-01'
enddate = '2022-02-22'
enddate = pd.to_datetime('today')

interval = '1h'
#easyband = pd.read_csv('easyband.csv')
easyband = pd.read_csv('allbands.csv')

#Columns=['Indicator','ALLTIME','CLOSERATIO','SMA100','SMA55']
#easyband = pd.DataFrame(easyband,columns=['Indicator','ATH','CLOSERATIO','RED','ORANGE','GREEN','Current'])
# Indicator,RED,ORANGE,GREEN,Current
easyband = pd.DataFrame(easyband,columns=['Indicator','RED','ORANGE','GREEN','Current'])

#easyband = easyband.loc[:,['Indicator','GREEN','GREENEASY']]

#EASY=float(easyband.loc[(easyband.Indicator=='BANDUSDT')].SMA100.values)
#print(easyband)
#EASY[0]
#easyband

In [60]:
entries = pd.read_csv('entries.csv')

entries = pd.DataFrame(entries, columns =['Indicator', 'lowest_entry']) 

#entries = pd.DataFrame(entrylist, columns =['Indicator', 'lowest_entry'],dtype = float) 
entries
entries.lowest_entry.astype(float)
indicator_list = entries['Indicator']
#indicator_list
entries
entries.to_csv("entries.csv")

exposurelist = pd.read_csv('exposure.csv')
exposure = pd.DataFrame(exposurelist, columns =['Indicator', 'exposure']) 
exposure
exposure.exposure.astype(float)
#indicator_list
#entries

#entries = entries.merge(exposure ,how="left",on="Indicator")
#entries

0      311.0
1     3500.0
2      527.0
3      700.0
4     6000.0
       ...  
59     500.0
60    1000.0
61     600.0
62    1000.0
63     400.0
Name: exposure, Length: 64, dtype: float64

In [61]:
#whitelist=whitelist.lstrip()
#whitelist = indicator_list
#array = whitelist.split(",")
array = indicator_list
wl = []
bl = []
blacklist = ""
whitelist = ""
symbols =[]
for i in range(0 ,len(array)):
    if (array[i].find('DOWN') != -1):
        bl.append(str(array[i]).lstrip())
        #print ("Contains given substring ")
        blacklist += str(array[i])
    elif (array[i].find('UP') != -1):
        bl.append(str(array[i]).lstrip())
        #print ("Contains given substring ")
        blacklist += str(array[i])

    else:
        wl.append(str(array[i]).lstrip().replace("/",""))
        whitelist +=str(array[i]).lstrip()
        #print ("Doesn't contains given substring")
    

    
#symbols to be used to get the prices from binance url
symbols = np.array(wl)


## get the data 
import os

live = pd.to_datetime('today')
start = pd.to_datetime(startdate)
end = pd.to_datetime(enddate)

root_url = 'https://api.binance.com/api/v1/klines'
symbol = 'STEEMETH'
interval = '1d'
url = root_url + '?symbol=' + symbol + '&interval=' + interval
print(url)


directory = 'BN-main'
table = []
   
        
        

for symbol in symbols:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval
        data = json.loads(requests.get(url).text)
        dflive = pd.DataFrame(data ,columns =['open_time',
              'o', 'h', 'l', 'c', 'v',
              'close_time', 'qav', 'num_trades',
              'taker_base_vol', 'taker_quote_vol', 'ignore'])
        dflive.open_time = pd.to_datetime(dflive['open_time'],unit='ms')
        dflive=dflive.loc[(dflive.open_time>start)]
        lastrow = dflive.iloc[-1]
        now= float(lastrow.c)
        alth = dflive.h.max()
        altl = dflive.l.min()
        
    
        ## lowest entiry
        lowest = float(entries.loc[(entries.Indicator==symbol)].lowest_entry.values)
        #print(lowest)
        easyseries = easyband.loc[(easyband.Indicator==symbol)].GREEN.values
        #print(easyseries)
        if easyseries.size>0:
                green = easyseries[0]
        else:
                green = 0
        oeasyseries = easyband.loc[(easyband.Indicator==symbol)].ORANGE.values
        #print(easyseries)
        if oeasyseries.size>0:
                orange = oeasyseries[0]
        else:
                orange = 0
 
        tuple =(symbol,alth,now,lowest,green,orange)
        table.append(tuple)
        #print("name is: ",symbol ,alth,altl)
        

table[0]
AllTime = pd.DataFrame(table,columns=['Indicator','ATH','Current','LOWEST','GREEN','ORANGE'])
AllTime.ATH=AllTime.ATH.astype(float)
#AllTime.ATL=AllTime.ATL.astype(float)
AllTime.Current=AllTime.Current.astype(float)
AllTime.LOWEST=AllTime.LOWEST.astype(float)
AllTime.GREEN=AllTime.GREEN.astype(float)
AllTime.ORANGE=AllTime.ORANGE.astype(float)




AllTime.dtypes

#AllTime['ALLTIME']=(AllTime['Current']/AllTime['ATH'])*100
#AllTime['MONTH']=(AllTime['Current']/AllTime['OMH'])*100
#AllTime['2MONTH']=(AllTime['Current']/AllTime['TMH'])*100
#AllTime['3MONTH']=(AllTime['Current']/AllTime['THMH'])*100
AllTime['DROP']=((AllTime['Current']-AllTime['LOWEST'])/AllTime['LOWEST'])*100
AllTime['GDROP']=((AllTime['GREEN']-AllTime['Current'])/AllTime['GREEN'])*100
AllTime['TDROP']=((AllTime['DROP']*AllTime['GDROP']))

#AllTime.sort_values(by=['ALLTIME'],ascending=True,inplace = True)
#AllTime.sort_values(by='ATH',ascending=False)
#AllTime.to_csv("alltimeHL.csv", mode="w")
#data.sort_values("Name", axis = 0, ascending = True,inplace = True, na_position ='last')

## Display the Data 
AllTime.shape



https://api.binance.com/api/v1/klines?symbol=STEEMETH&interval=1d


(64, 9)

In [62]:
## Display the Data coins run last  Month

#drop = AllTime.loc[(AllTime['MONTH'] < 35)]
#drop = AllTime.loc[(AllTime['MONTH'] < 70)]
#drop = AllTime.loc[(AllTime['MONTH'] < 80)&(AllTime['2MONTH'] < 70)&(AllTime['ALLTIME'] < 40)]
drop = AllTime.loc[(AllTime['Current'] <= AllTime['LOWEST'])]

#sort = drop.sort_values(by=['EASY'],ascending=False,inplace = True)
sort = drop.sort_values(by=['GDROP','DROP'],ascending=[False,True],inplace = True)
sort = drop.sort_values(by=['TDROP'],ascending=True,inplace = True)

#drop = drop.loc[(drop['ALLTIME'] < drop['3MONTH'])]
display = drop[['Indicator','TDROP','GREEN','DROP','GDROP','LOWEST','Current']]

#print(display.iloc[0:60,0:14])
#print(display.iloc[60:110,])
#print(display.iloc[110:160,])
print("\b Fix Lowest entry ")

display = display.merge(exposure ,how="left",on="Indicator")
#display1 = display.sort_values(by=['exposure'],ascending=True,inplace = True)


display.to_csv("fixentries.csv")

display

 Fix Lowest entry 


/home/haytham/anaconda3/envs/ML20/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
0,LUNAUSDT,-9999.948133,37.298096,-99.999615,99.999866,13.000000,0.000050,4000.0
1,MIRUSDT,-6137.918496,0.686223,-90.562100,67.775797,2.343000,0.221130,85.0
2,PERPUSDT,-2734.349044,2.550238,-49.099099,55.690412,2.220000,1.130000,300.0
3,SUSHIUSDT,-1902.136312,1.946262,-61.571429,30.893165,3.500000,1.345000,2730.0
4,RAYUSDT,-1536.590424,1.594333,-54.280000,28.308593,2.500000,1.143000,100.0
5,HNTUSDT,-1222.580715,11.434524,-41.798561,29.249349,13.900000,8.090000,1250.0
6,FILUSDT,-1148.851006,10.153810,-59.100000,19.439103,20.000000,8.180000,2000.0
7,DENTUSDT,-1110.475898,0.001313,-57.015047,19.476892,0.002459,0.001057,385.0
8,GRTUSDT,-998.776726,0.198471,-49.841972,20.038868,0.316400,0.158700,1700.0
9,CRVUSDT,-989.572144,1.399381,-47.688679,20.750672,2.120000,1.109000,323.0


## Green Drop


In [63]:
## Green Drop
display = display.loc[:]
#display = display.loc[display.DROP <= -30]
display.sort_values(by=['GDROP'],ascending=False,inplace = True)

#display = display.loc[display.Current < display.GREEN*.9]

display

,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
0,LUNAUSDT,-9999.948133,37.298096,-99.999615,99.999866,13.000000,0.000050,4000.0
1,MIRUSDT,-6137.918496,0.686223,-90.562100,67.775797,2.343000,0.221130,85.0
2,PERPUSDT,-2734.349044,2.550238,-49.099099,55.690412,2.220000,1.130000,300.0
3,SUSHIUSDT,-1902.136312,1.946262,-61.571429,30.893165,3.500000,1.345000,2730.0
5,HNTUSDT,-1222.580715,11.434524,-41.798561,29.249349,13.900000,8.090000,1250.0
4,RAYUSDT,-1536.590424,1.594333,-54.280000,28.308593,2.500000,1.143000,100.0
9,CRVUSDT,-989.572144,1.399381,-47.688679,20.750672,2.120000,1.109000,323.0
8,GRTUSDT,-998.776726,0.198471,-49.841972,20.038868,0.316400,0.158700,1700.0
7,DENTUSDT,-1110.475898,0.001313,-57.015047,19.476892,0.002459,0.001057,385.0
6,FILUSDT,-1148.851006,10.153810,-59.100000,19.439103,20.000000,8.180000,2000.0


## Max entry Drop


In [64]:
## Max entry Drop
display = display.loc[:]
#display = display.loc[display.DROP <= -30]
display.sort_values(by=['DROP'],ascending=True,inplace = True)

#display = display.loc[display.Current < display.GREEN*.9]

display

,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
0,LUNAUSDT,-9999.948133,37.298096,-99.999615,99.999866,13.000000,0.000050,4000.0
1,MIRUSDT,-6137.918496,0.686223,-90.562100,67.775797,2.343000,0.221130,85.0
3,SUSHIUSDT,-1902.136312,1.946262,-61.571429,30.893165,3.500000,1.345000,2730.0
44,CAKEUSDT,887.833083,3.971286,-59.386121,-14.950178,11.240000,4.565000,4000.0
6,FILUSDT,-1148.851006,10.153810,-59.100000,19.439103,20.000000,8.180000,2000.0
7,DENTUSDT,-1110.475898,0.001313,-57.015047,19.476892,0.002459,0.001057,385.0
4,RAYUSDT,-1536.590424,1.594333,-54.280000,28.308593,2.500000,1.143000,100.0
12,THETAUSDT,-855.228899,1.520738,-54.250000,15.764588,2.800000,1.281000,500.0
50,FLUXUSDT,inf,0.000000,-54.169402,-inf,1.523000,0.698000,224.0
15,MATICUSDT,-710.921675,0.757310,-53.142857,13.377558,1.400000,0.656000,8000.0


In [65]:
## GT drop
print("\b Enries on easy band")

display1 = display.loc[:]
display1['GT']=display1['TDROP']*display1['exposure']
display1.sort_values(by=['GT'],ascending=True,inplace = True)
display1

 Enries on easy band


,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure,GT
0,LUNAUSDT,-9999.948133,37.298096,-99.999615,99.999866,13.000000,0.000050,4000.0,-3.999979e+07
15,MATICUSDT,-710.921675,0.757310,-53.142857,13.377558,1.400000,0.656000,8000.0,-5.687373e+06
3,SUSHIUSDT,-1902.136312,1.946262,-61.571429,30.893165,3.500000,1.345000,2730.0,-5.192832e+06
6,FILUSDT,-1148.851006,10.153810,-59.100000,19.439103,20.000000,8.180000,2000.0,-2.297702e+06
20,LINKUSDT,-364.556502,7.745952,-41.166667,8.855624,12.000000,7.060000,6000.0,-2.187339e+06
8,GRTUSDT,-998.776726,0.198471,-49.841972,20.038868,0.316400,0.158700,1700.0,-1.697920e+06
5,HNTUSDT,-1222.580715,11.434524,-41.798561,29.249349,13.900000,8.090000,1250.0,-1.528226e+06
2,PERPUSDT,-2734.349044,2.550238,-49.099099,55.690412,2.220000,1.130000,300.0,-8.203047e+05
28,KSMUSDT,-71.159228,82.347619,-30.086957,2.365119,115.000000,80.400000,7800.0,-5.550420e+05
14,RENUSDT,-787.881339,0.172983,-51.854283,15.194142,0.304700,0.146700,684.0,-5.389108e+05


In [66]:
#display1['Value']=(display['DROP']/100)*display['exposure']
#display1

display = display1.loc[:]
#display['RED']=display['ORANGE']*2
display['Value']=(display['DROP']/100)*display['exposure']
sort = display.sort_values(by=['Value'],ascending=True,inplace = True)

#display['Ratio']=display['ATH']/display['Current']
#display['ORatio']=display['ORANGE']/display['Current']
#display = display.loc[display['Current'] <= display['GREEN']]
display


In [67]:
#del display ,sort, AllTime

In [68]:
#AllTime